# Random Number Generation for JIT Compilation

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brainpy/brainpy/blob/master/docs_version2/tutorial_math/random_number_generation.ipynb)
[![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/brainpy/brainpy/blob/master/docs_version2/tutorial_math/random_number_generation.ipynb)

[Chaoming Wang](mailto:chao.brain@qq.com)

Although ``brainpy.math.random`` is designed to be seamlessly compatible with ``numpy.random``, there are still some differences under the context of JIT compilation.

In this section, we are going to talk about how to program a JIT-compatible code with ``brainpy.math.random``.

In [28]:
import brainpy as bp
import brainpy.math as bm
import numpy as np

# bm.set_platform('cpu')

In [29]:
bp.__version__

'3.0.0'

## Using ``bm.random`` outside functions to JIT

Using ``brainpy.math.random`` outside of functions to JIT is the same as using ``numpy.random``.

This usage corresponds to the cases that generating random data for further processing. For example,

In [30]:
bm.random.rand(10)

Array([0.3089733 , 0.89542127, 0.02714849, 0.94584775, 0.77089345,
       0.32520366, 0.69955456, 0.37229824, 0.43667316, 0.02874231],      dtype=float32)

In [31]:
np.random.rand(10)

array([0.18021362, 0.57020223, 0.91773743, 0.32111789, 0.592986  ,
       0.32347693, 0.66178114, 0.11078982, 0.20402456, 0.80672322])

When you are using API functions in ``brainpy.math.random``, actually you are calling functions in a default ``RandomState``. Similarly, ``numpy.random`` also has a default ``RandomState``. Calling a random function in ``numpy.random`` module corresponds to calling the random function in this default NumPy ``RandomState``.

In [32]:
bm.random.DEFAULT

RandomState([2354496134 1053393286])

## Using ``bm.random`` inside a function to JIT

If you are using random sampling in a JIT function, **there are things you need to pay attention to**. Otherwise, the error is likely to raise.

As I have stated above, ``brainpy.math.random`` functions are using the default ``RandomState``. A ``RandomState`` is an instance of brainpy [Variable](./arrays_and_variables.ipynb), denoting that it has values to change after calling any its built-in random function. What's changing is the `key` of a ``RandomState``. For instance,

In [33]:
bm.random.rand(1)
print('Now, the DEFAULT is', bm.random.DEFAULT)

Now, the DEFAULT is RandomState([3949348841  188664647])


In [34]:
bm.random.rand(1)
print('Now, the DEFAULT is', bm.random.DEFAULT)

Now, the DEFAULT is RandomState([1197037675  819676900])


Therefore, if you do not specify this DEFAULT ``RandomState`` you are using, repeatedly calling random functions in ``brainpy.math.random`` module will not get what you want, because its `key` cannot be updated. For instance,

In [35]:
@bm.jit
def get_data():
    return bm.random.random(2)

In [36]:
get_data()

Array([0.6563661 , 0.82885206], dtype=float32)

In [37]:
get_data()

Array([0.8161644, 0.5287926], dtype=float32)

A correct way is explicitly declaring you are using this DEFAULT variable in the JIT transformation.

In [38]:
bm.random.seed()

In [39]:
from functools import partial

@partial(bm.jit)
def get_data_v2():
    return bm.random.random(2)

In [40]:
get_data_v2()

Array([0.18351912, 0.07486355], dtype=float32)

In [41]:
get_data_v2()

Array([0.6289803, 0.797102 ], dtype=float32)

Or, declare the function as a `BrainPyObject`, then use ``jit()``.

In [42]:
@bm.jit
def get_data_v3():
    return bm.random.random(2)

In [43]:
get_data_v3()

Array([0.64915967, 0.37961698], dtype=float32)

In [44]:
get_data_v3()

Array([0.28215742, 0.5177479 ], dtype=float32)

## Using ``RandomState`` for objects to JIT

Another way I recommend is using instances of ``RandomState`` for objects to JIT. For example, you can initialize a ``RandomState`` in the ``__init__()`` function, then using the initialized ``RandomState`` anywhere.

In [45]:
class MyOb(bp.BrainPyObject):
    def __init__(self):
        super().__init__()
        self.rng = bm.random.RandomState(123)

    def __call__(self):
        size = (50, 100)
        u = self.rng.random(size)
        v = self.rng.uniform(size=size)
        z = bm.sqrt(-2 * bm.log(u)) * bm.cos(2 * bm.pi * v)
        return z

In [46]:
ob = bm.jit(MyOb())

In [47]:
ob()

Array([[ 1.714168  , -0.93755597, -1.566088  , ..., -2.300819  ,
        -0.3244472 , -1.0213778 ],
       [ 0.22194532,  0.86869   , -1.8543271 , ...,  0.6031633 ,
         0.5587094 ,  0.17947507],
       [-0.04175869, -0.25695515,  0.41855374, ...,  0.7097617 ,
        -1.0369484 , -2.6332123 ],
       ...,
       [-0.5513834 , -1.2007083 , -0.21716705, ...,  2.4005718 ,
        -1.97702   , -1.9229662 ],
       [ 1.9620371 , -1.030306  ,  0.5033778 , ...,  1.0114168 ,
         0.5670819 ,  0.3017683 ],
       [-1.576822  , -1.2365927 ,  0.87836343, ..., -1.5918341 ,
         0.7913105 , -1.8031495 ]], dtype=float32)

Note that any ``Variable`` instance which can be directly accessed by ``self.`` is able to be automatically found by brainpy's JIT transformation functions.
Therefore, in this case, we do not need to pass the ``rng`` into the `dyn_vars` in ``bm.jit()`` function.